In [78]:
import pandas as pd
from io import BytesIO
import requests

r = requests.get("https://docs.google.com/spreadsheet/ccc?key=1JyGlqmLg9k7UubOw-V_CC8McRZxn2PtknvsrMIxvLGk&output=csv")
data = r.content

df = pd.read_csv(BytesIO(data), index_col=0, skiprows=2)

df.head()

,Local Id,Source,Type,AR,ALG,DA,GEO,ARG,FAD,PCT,...,PROB,PRS,CC,TRI,CG,MG,WP,2WP,VC,AQ
Global Id,,,,,,,,,,,,,,,,,,,,,
1,7.01,M,QC,x,NaN,NaN,NaN,x,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.02,M,QC,x,NaN,NaN,NaN,x,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7.03,M,QC,x,NaN,NaN,NaN,x,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.04,M,NE,x,NaN,NaN,NaN,x,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7.05,M,QC,x,NaN,NaN,NaN,x,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
#dictOfWords = { i : False for i in df.columns }
#dictOfWords
repl_nan = {
 'AR': False,
 'ALG': False,
 'DA': False,
 'GEO': False,
 'ARG': False,
 'FAD': False,
 'PCT': False,
 'NP': False,
 'DAP': False,
 'EAR': False,
 'EAF': False,
 'IAV': False,
 'FFS': False,
 'ROW': False,
 'RAT': False,
 'CT': False,
 'DST': False,
 'PROB': False,
 'DI': False,
 'PRS': False,
 'CC': False,
 'TRI': False,
 'CG': False,
 'MG': False,
 'WP': False,
 '2WP': False,
 'VC': False,
 'AQ': False
}

repl_checked = {
 'AR': 'x',
 'ALG': 'x',
 'DA': 'x',
 'GEO': 'x',
 'ARG': 'x',
 'FAD': 'x',
 'PCT': 'x',
 'NP': 'x',
 'DAP': 'x',
 'EAR': 'x',
 'EAF': 'x',
 'IAV': 'x',
 'FFS': 'x',
 'ROW': 'x',
 'RAT': 'x',
 'CT': 'x',
 'DST': 'x',
 'PROB': 'x',
 'DI': 'x',
 'PRS': 'x',
 'CC': 'x',
 'TRI': 'x',
 'CG': 'x',
 'MG': 'x',
 'WP': 'x',
 '2WP': 'x',
 'VC': 'x',
 'AQ': 'x'
}
scrubbed = df.fillna(value=repl_nan).replace(repl_checked, True)
scrubbed.head()

,Local Id,Source,Type,AR,ALG,DA,GEO,ARG,FAD,PCT,...,PROB,PRS,CC,TRI,CG,MG,WP,2WP,VC,AQ
Global Id,,,,,,,,,,,,,,,,,,,,,
1,7.01,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,7.02,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,7.03,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,7.04,M,NE,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
5,7.05,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [83]:
categorical_cols = scrubbed.columns[3:7]
ctotals = { col:scrubbed[col].where(scrubbed[col]==True).count() for col in categorical_cols }
ctotals

{'AR': 233, 'ALG': 169, 'DA': 115, 'GEO': 132}

In [82]:
skillset_cols = scrubbed.columns[7:-4]
    
stotals = { col:scrubbed[col].where(scrubbed[col]==True).count() for col in skillset_cols }
stotals

{'ARG': 29,
 'FAD': 40,
 'PCT': 54,
 'NP': 44,
 'DAP': 34,
 'EAR': 38,
 'EAF': 35,
 'IAV': 44,
 'FFS': 58,
 'ROW': 30,
 'RAT': 41,
 'CT': 37,
 'DST': 31,
 'PROB': 47,
 'PRS': 22,
 'CC': 26,
 'TRI': 43,
 'CG': 24,
 'MG': 18}

In [84]:
misc_cols = scrubbed.columns[-4:]
stotals = { col:scrubbed[col].where(scrubbed[col]==True).count() for col in misc_cols }
stotals

{'WP': 36, '2WP': 25, 'VC': 20, 'AQ': 42}

In [44]:
scrubbed.where(scrubbed["ARG"]==True).count()

Local Id    29
Source      29
Type        29
AR          29
ALG         29
DA          29
GEO         29
ARG         29
FAD         29
PCT         29
NP          29
DAP         29
EAR         29
EAF         29
IAV         29
FFS         29
ROW         29
RAT         29
CT          29
DST         29
PROB        29
DI          29
PRS         29
CC          29
TRI         29
CG          29
MG          29
WP          29
2WP         29
VC          29
AQ          29
dtype: int64